# <center>Machine Learning Project</center>

** **
## <center>*03.7 - Neural Networks*</center>

** **

The members of the `team` are:
- Ana Farinha - 20211514
- Francisco Capontes - 20211692
- Sofia Gomes - 20240848
- Rui Lourenço - 2021639



In [1]:
# Import libraries
import pandas as pd
import numpy as np

#make the split here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import StratifiedKFold
import time


from utils import *
from utils_dicts import *

## <span style="color:salmon"> 1. Import Dataset </span> 

In [2]:
# Import dataset
df = pd.read_csv('preprocessed_data/train_data.csv', index_col="Claim Identifier")
df

,Accident Date,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Known Accident Date,Known Assembly Date,Known C-2 Date,Known C-3 Date,...,Zip_Code_Hash_13,Zip_Code_Hash_14,Zip_Code_Hash_15,Zip_Code_Hash_16,Zip_Code_Hash_17,Zip_Code_Hash_18,Zip_Code_Hash_19,Claim Injury Type Encoded,Accident_Season_Sin,Accident_Season_Cos
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5393875,2019-12-30,31.0,NaN,1988.0,0.0,1.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000e+00,1.000000e+00
5393091,2019-08-30,46.0,1745.93,1973.0,4.0,4.0,1,1,1,1,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,3,1.224647e-16,-1.000000e+00
5393889,2019-12-06,40.0,1434.80,1979.0,0.0,6.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,3,-1.000000e+00,-1.836970e-16
5393887,2019-12-30,61.0,NaN,1958.0,0.0,1.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000e+00,1.000000e+00
5393863,2019-12-26,67.0,NaN,1952.0,0.0,5.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.000000e+00,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6165265,2022-12-14,35.0,NaN,1987.0,0.0,4.0,1,1,1,0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1,-1.000000e+00,-1.836970e-16
6165285,2022-12-15,33.0,NaN,1989.0,0.0,6.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,-1.000000e+00,-1.836970e-16
6165506,2022-12-13,61.0,991.08,1961.0,1.0,3.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,3,-1.000000e+00,-1.836970e-16


In [3]:
# Import dataset
test_df = pd.read_csv('./preprocessed_data/test_data.csv', index_col = 'Claim Identifier')
test_df

,Accident Date,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Known Accident Date,Known Assembly Date,Known C-2 Date,Known C-3 Date,...,Zip_Code_Hash_12,Zip_Code_Hash_13,Zip_Code_Hash_14,Zip_Code_Hash_15,Zip_Code_Hash_16,Zip_Code_Hash_17,Zip_Code_Hash_18,Zip_Code_Hash_19,Accident_Season_Sin,Accident_Season_Cos
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
6165911,2022-12-24,19.0,NaN,2003.0,0.0,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.000000e+00
6166141,2022-11-20,19.0,NaN,2003.0,0.0,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000000e+00,-1.836970e-16
6165907,2022-12-26,59.0,NaN,1963.0,0.0,0,1,1,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000e+00,1.000000e+00
6166047,2022-12-28,55.0,NaN,1967.0,0.0,6,1,1,1,0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.000000e+00
6166102,2022-12-20,25.0,NaN,1997.0,0.0,5,1,1,1,0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.000000e+00,-1.836970e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6553137,2012-09-12,52.0,NaN,1960.0,0.0,5,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.224647e-16,-1.000000e+00
6553119,2024-05-22,59.0,NaN,1965.0,0.0,1,1,1,0,1,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.000000e+00,6.123234e-17
6553542,2024-05-06,45.0,NaN,1979.0,0.0,5,1,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.000000e+00,6.123234e-17


## <span style="color:salmon"> 2. Prepare Dataset </span> 

Define y as a target "Claim Injury Type Encoded" and X with all the other columns

In [4]:
X = df.drop(["Claim Injury Type Encoded"], axis = 1)
y = df["Claim Injury Type Encoded"]

Split the data and the target to X_train, X_test, y_train, y_test, where test_size should be equal to 0.25, stratify equal to target and shuffle equal to True

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.75, stratify = y, shuffle = True)

Define X_test

In [6]:
X_test = test_df.copy()

Check missing values in each dataset:

In [7]:
X_train.isna().sum().sort_values()[X_train.isna().sum() != 0]

Birth Year               533
Age at Injury            533
Average Weekly Wage    91224
dtype: int64

In [8]:
X_val.isna().sum().sort_values()[X_val.isna().sum() != 0]

Birth Year               1653
Age at Injury            1653
Average Weekly Wage    272875
dtype: int64

In [9]:
X_test.isna().sum().sort_values()[X_test.isna().sum() != 0]

Birth Year                982
Age at Injury             982
Average Weekly Wage    335753
dtype: int64

In [10]:
NA_imputer(X_train,X_val)
create_new_features(X_train,X_val)

In [11]:
NA_imputer(X,X_test)
create_new_features(X,X_test)

In [13]:
scaler_train = StandardScaler()
X_train[numerical_features] = scaler_train.fit_transform(X_train[numerical_features])
X_val[numerical_features] = scaler_train.transform(X_val[numerical_features])
scaler_predict = StandardScaler()
X[numerical_features] = scaler_predict.fit_transform(X[numerical_features])
X_test[numerical_features] = scaler_predict.transform(X_test[numerical_features])

## <span style="color:salmon"> 3. CatBoosted </span> 

In [14]:
config = {'iterations': 500, 'learning_rate': 0.15, 'depth': 6, 'l2_leaf_reg': 9}

In [16]:
model_cb = CatBoostClassifier(
        iterations=config["iterations"],
        learning_rate=config["learning_rate"],
        depth=config["depth"],
        l2_leaf_reg=config["l2_leaf_reg"],
        loss_function="MultiClass", 
        eval_metric="MultiClass",  
        custom_metric=['F1'], 
        verbose=0
    )
model_cb.fit(X_train,y_train)

In [17]:
y_train_pred = model_cb.predict(X_train)

In [18]:
y_val_pred = model_cb.predict(X_val)

#### <span style="color:salmon"> 3.1  Evaluate the model </span> 


In [23]:
class_mapping = {
    0:'1. CANCELLED', 
    1:'2. NON-COMP',
    2:'3. MED ONLY', 
    3:'4. TEMPORARY',
    4:'5. PPD SCH LOSS', 
    5:'6. PPD NSL', 
    6:'7. PTD', 
    7:'8. DEATH'
}

# Use the values from class_mapping as the target names
target_names = list(class_mapping.values())

Compute confusion matrix to evaluate the accuracy of a classification

In [24]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, target_names=target_names))

Confusion Matrix:
[[ 1223  1771    54    67     4     0     0     0]
 [  463 71495   302   490    19     0     0     0]
 [   18  9099  1723  5775   609     0     0     2]
 [   17  2003   465 33007  1634     0     0     1]
 [    0   179   185  4978  6727     1     0     0]
 [    0     5    17   874   140    17     0     0]
 [    0     0     0    20     2     0     2     0]
 [    0    14     6    34     0     0     0    64]]

Classification Report:
                 precision    recall  f1-score   support

   1. CANCELLED       0.71      0.39      0.51      3119
    2. NON-COMP       0.85      0.98      0.91     72769
    3. MED ONLY       0.63      0.10      0.17     17226
   4. TEMPORARY       0.73      0.89      0.80     37127
5. PPD SCH LOSS       0.74      0.56      0.63     12070
     6. PPD NSL       0.94      0.02      0.03      1053
         7. PTD       1.00      0.08      0.15        24
       8. DEATH       0.96      0.54      0.69       118

       accuracy                   

In [25]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=target_names))


Confusion Matrix:
[[  3458   5522    168    199      7      0      0      2]
 [  1627 213487   1329   1759    102      0      0      5]
 [    47  27496   3843  18280   2003      2      0      9]
 [    56   6267   1865  97342   5806      8      0     36]
 [     0    563    546  16538  18558      2      0      3]
 [     0      3     37   2639    476      3      0      0]
 [     0      0      1     63      8      0      0      1]
 [     1     73     13    198      5      0      0     62]]

Classification Report:


C:\Users\ruipb\Desktop\Masters\Projects\Machine_Learning_NOVAIMS\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                 precision    recall  f1-score   support

   1. CANCELLED       0.67      0.37      0.48      9356
    2. NON-COMP       0.84      0.98      0.91    218309
    3. MED ONLY       0.49      0.07      0.13     51680
   4. TEMPORARY       0.71      0.87      0.78    111380
5. PPD SCH LOSS       0.69      0.51      0.59     36210
     6. PPD NSL       0.20      0.00      0.00      3158
         7. PTD       0.00      0.00      0.00        73
       8. DEATH       0.53      0.18      0.26       352

       accuracy                           0.78    430518
      macro avg       0.52      0.37      0.39    430518
   weighted avg       0.74      0.78      0.74    430518



C:\Users\ruipb\Desktop\Masters\Projects\Machine_Learning_NOVAIMS\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ruipb\Desktop\Masters\Projects\Machine_Learning_NOVAIMS\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## <span style="color:salmon"> 4. Test Predictions </span> 

Make validation predictions:

In [ ]:
y_test_pred = model_cb.predict(X_test)

In [ ]:
test_id = model_cb.index

In [ ]:
submission_df = pd.DataFrame({
    'Claim Identifier': test_id,
    'Claim Injury Type': y_test_pred
})